Aim of the project: Predict Walmart Sales 

Let's start by importing our data

In [1]:
import pandas as pd 
import numpy as np
import os 
os.getcwd()


'c:\\Users\\hamoud\\Desktop\\Walmart'

In [2]:
test = pd.read_csv("./Data/test.csv")
sampleSubmission = pd.read_csv("./Data/sampleSubmission.csv")

In [3]:
test.head()

,Store,Dept,Date,IsHoliday
0,1,1,2012-11-02,False
1,1,1,2012-11-09,False
2,1,1,2012-11-16,False
3,1,1,2012-11-23,True
4,1,1,2012-11-30,False


In [4]:
sampleSubmission.head()

,Id,Weekly_Sales
0,1_1_2012-11-02,0
1,1_1_2012-11-09,0
2,1_1_2012-11-16,0
3,1_1_2012-11-23,0
4,1_1_2012-11-30,0


In [5]:
train = pd.read_csv("./Data/train.csv")
features = pd.read_csv("./Data/features.csv")
stores = pd.read_csv("./Data/stores.csv")

In [6]:
train.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True
2,1,1,2010-02-19,41595.55,False
3,1,1,2010-02-26,19403.54,False
4,1,1,2010-03-05,21827.90,False


In [7]:
features.head()

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday
0,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False
1,1,2010-02-12,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True
2,1,2010-02-19,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False
3,1,2010-02-26,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False
4,1,2010-03-05,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False


In [8]:
stores.head()

,Store,Type,Size
0,1,A,151315
1,2,A,202307
2,3,B,37392
3,4,A,205863
4,5,B,34875


In [9]:
train.info()

<class 'pandas.DataFrame'>
RangeIndex: 421570 entries, 0 to 421569
Data columns (total 5 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Store         421570 non-null  int64  
 1   Dept          421570 non-null  int64  
 2   Date          421570 non-null  str    
 3   Weekly_Sales  421570 non-null  float64
 4   IsHoliday     421570 non-null  bool   
dtypes: bool(1), float64(1), int64(2), str(1)
memory usage: 17.3 MB


In [10]:
features.info()

<class 'pandas.DataFrame'>
RangeIndex: 8190 entries, 0 to 8189
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Store         8190 non-null   int64  
 1   Date          8190 non-null   str    
 2   Temperature   8190 non-null   float64
 3   Fuel_Price    8190 non-null   float64
 4   MarkDown1     4032 non-null   float64
 5   MarkDown2     2921 non-null   float64
 6   MarkDown3     3613 non-null   float64
 7   MarkDown4     3464 non-null   float64
 8   MarkDown5     4050 non-null   float64
 9   CPI           7605 non-null   float64
 10  Unemployment  7605 non-null   float64
 11  IsHoliday     8190 non-null   bool   
dtypes: bool(1), float64(9), int64(1), str(1)
memory usage: 791.9 KB


In [11]:
stores.info()

<class 'pandas.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   Store   45 non-null     int64
 1   Type    45 non-null     str  
 2   Size    45 non-null     int64
dtypes: int64(2), str(1)
memory usage: 1.2 KB


We will start by doing the necessary merges:
merge stores and features based on Store column

In [12]:
feature_store_merged = features.merge(stores, how="inner",on = "Store")
feature_store_merged.head()

,Store,Date,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,IsHoliday,Type,Size
0,1,2010-02-05,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,False,A,151315
1,1,2010-02-12,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,True,A,151315
2,1,2010-02-19,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,False,A,151315
3,1,2010-02-26,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,False,A,151315
4,1,2010-03-05,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,False,A,151315


In [13]:
train.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday
0,1,1,2010-02-05,24924.50,False
1,1,1,2010-02-12,46039.49,True
2,1,1,2010-02-19,41595.55,False
3,1,1,2010-02-26,19403.54,False
4,1,1,2010-03-05,21827.90,False


Now, we can merge the training and testing dataset with the feature_store_merged on the Store/Date/IsHoliday Columns
Then we will sort the data  based on the Store, then the Departement and then the Date/Week.And finally we will 
reset the indexes

In [14]:
train_final_merged = train.merge(feature_store_merged, on = ["Store","Date","IsHoliday"])
train_final_merged = train_final_merged.sort_values(by= ["Store","Dept","Date"])
train_final_merged = train_final_merged.reset_index(drop=True)
train_final_merged.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size
0,1,1,2010-02-05,24924.50,False,42.31,2.572,NaN,NaN,NaN,NaN,NaN,211.096358,8.106,A,151315
1,1,1,2010-02-12,46039.49,True,38.51,2.548,NaN,NaN,NaN,NaN,NaN,211.242170,8.106,A,151315
2,1,1,2010-02-19,41595.55,False,39.93,2.514,NaN,NaN,NaN,NaN,NaN,211.289143,8.106,A,151315
3,1,1,2010-02-26,19403.54,False,46.63,2.561,NaN,NaN,NaN,NaN,NaN,211.319643,8.106,A,151315
4,1,1,2010-03-05,21827.90,False,46.50,2.625,NaN,NaN,NaN,NaN,NaN,211.350143,8.106,A,151315


In [15]:
test_final_merged = test.merge(feature_store_merged, how="inner", on = ["Store","Date","IsHoliday"])
test_final_merged = test_final_merged.sort_values(by= ["Store","Dept","Date"])
test_final_merged = test_final_merged.reset_index(drop=True)
test_final_merged.head()

,Store,Dept,Date,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size
0,1,1,2012-11-02,False,55.32,3.386,6766.44,5147.70,50.82,3639.90,2737.42,223.462779,6.573,A,151315
1,1,1,2012-11-09,False,61.24,3.314,11421.32,3370.89,40.28,4646.79,6154.16,223.481307,6.573,A,151315
2,1,1,2012-11-16,False,52.92,3.252,9696.28,292.10,103.78,1133.15,6612.69,223.512911,6.573,A,151315
3,1,1,2012-11-23,True,56.23,3.211,883.59,4.17,74910.32,209.91,303.32,223.561947,6.573,A,151315
4,1,1,2012-11-30,False,52.34,3.207,2460.03,NaN,3838.35,150.57,6966.34,223.610984,6.573,A,151315


In [16]:
test_final_merged.info()

<class 'pandas.DataFrame'>
RangeIndex: 115064 entries, 0 to 115063
Data columns (total 15 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   Store         115064 non-null  int64  
 1   Dept          115064 non-null  int64  
 2   Date          115064 non-null  str    
 3   IsHoliday     115064 non-null  bool   
 4   Temperature   115064 non-null  float64
 5   Fuel_Price    115064 non-null  float64
 6   MarkDown1     114915 non-null  float64
 7   MarkDown2     86437 non-null   float64
 8   MarkDown3     105235 non-null  float64
 9   MarkDown4     102176 non-null  float64
 10  MarkDown5     115064 non-null  float64
 11  CPI           76902 non-null   float64
 12  Unemployment  76902 non-null   float64
 13  Type          115064 non-null  str    
 14  Size          115064 non-null  int64  
dtypes: bool(1), float64(9), int64(3), str(2)
memory usage: 13.6 MB


***
Now, let's change the dtype of the Date coloumn to datetime in both sets, and create new data for them (year/month/day/week)
***

In [17]:
train_final_merged["Date"] = pd.to_datetime(train_final_merged["Date"])
train_final_merged["Date"]

0        2010-02-05
1        2010-02-12
2        2010-02-19
3        2010-02-26
4        2010-03-05
            ...    
421565   2012-09-28
421566   2012-10-05
421567   2012-10-12
421568   2012-10-19
421569   2012-10-26
Name: Date, Length: 421570, dtype: datetime64[us]

In [18]:
train_final_merged.info()

<class 'pandas.DataFrame'>
RangeIndex: 421570 entries, 0 to 421569
Data columns (total 16 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   Store         421570 non-null  int64         
 1   Dept          421570 non-null  int64         
 2   Date          421570 non-null  datetime64[us]
 3   Weekly_Sales  421570 non-null  float64       
 4   IsHoliday     421570 non-null  bool          
 5   Temperature   421570 non-null  float64       
 6   Fuel_Price    421570 non-null  float64       
 7   MarkDown1     150681 non-null  float64       
 8   MarkDown2     111248 non-null  float64       
 9   MarkDown3     137091 non-null  float64       
 10  MarkDown4     134967 non-null  float64       
 11  MarkDown5     151432 non-null  float64       
 12  CPI           421570 non-null  float64       
 13  Unemployment  421570 non-null  float64       
 14  Type          421570 non-null  str           
 15  Size          421570 non-nul

Now, let's add new columns from the Date Column, namely year,month,day and week of the year and week of the year.

In [19]:
train_final_merged["Year"] = train_final_merged["Date"].dt.year
train_final_merged["Month"] = train_final_merged["Date"].dt.month
train_final_merged["Day"] = train_final_merged["Date"].dt.day
train_final_merged["Week_Of_Year"] = train_final_merged["Date"].dt.isocalendar().week.astype(int)
train_final_merged["Week_Of_Month"] = ((train_final_merged["Date"].dt.day-1) // 7 + 1)


| Day   | After -1 | //7 |
| ----- | -------- | --- |
| 1–7   | 0–6      | 0   |
| 8–14  | 7–13     | 1   |
| 15–21 | 14–20    | 2   |
| 22–28 | 21–27    | 3   |
| 29–31 | 28–30    | 4   |


After we add 1

| Day   | WeekOfMonth |
| ----- | ----------- |
| 1–7   | 1           |
| 8–14  | 2           |
| 15–21 | 3           |
| 22–28 | 4           |
| 29–31 | 5           |


In [20]:
train_final_merged.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,...,MarkDown5,CPI,Unemployment,Type,Size,Year,Month,Day,Week_Of_Year,Week_Of_Month
0,1,1,2010-02-05,24924.50,False,42.31,2.572,NaN,NaN,NaN,...,NaN,211.096358,8.106,A,151315,2010,2,5,5,1
1,1,1,2010-02-12,46039.49,True,38.51,2.548,NaN,NaN,NaN,...,NaN,211.242170,8.106,A,151315,2010,2,12,6,2
2,1,1,2010-02-19,41595.55,False,39.93,2.514,NaN,NaN,NaN,...,NaN,211.289143,8.106,A,151315,2010,2,19,7,3
3,1,1,2010-02-26,19403.54,False,46.63,2.561,NaN,NaN,NaN,...,NaN,211.319643,8.106,A,151315,2010,2,26,8,4
4,1,1,2010-03-05,21827.90,False,46.50,2.625,NaN,NaN,NaN,...,NaN,211.350143,8.106,A,151315,2010,3,5,9,1


In [21]:
test_final_merged["Date"] = pd.to_datetime(test_final_merged["Date"])
test_final_merged["Date"]

0        2012-11-02
1        2012-11-09
2        2012-11-16
3        2012-11-23
4        2012-11-30
            ...    
115059   2013-06-28
115060   2013-07-05
115061   2013-07-12
115062   2013-07-19
115063   2013-07-26
Name: Date, Length: 115064, dtype: datetime64[us]

In [22]:
test_final_merged.info()

<class 'pandas.DataFrame'>
RangeIndex: 115064 entries, 0 to 115063
Data columns (total 15 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   Store         115064 non-null  int64         
 1   Dept          115064 non-null  int64         
 2   Date          115064 non-null  datetime64[us]
 3   IsHoliday     115064 non-null  bool          
 4   Temperature   115064 non-null  float64       
 5   Fuel_Price    115064 non-null  float64       
 6   MarkDown1     114915 non-null  float64       
 7   MarkDown2     86437 non-null   float64       
 8   MarkDown3     105235 non-null  float64       
 9   MarkDown4     102176 non-null  float64       
 10  MarkDown5     115064 non-null  float64       
 11  CPI           76902 non-null   float64       
 12  Unemployment  76902 non-null   float64       
 13  Type          115064 non-null  str           
 14  Size          115064 non-null  int64         
dtypes: bool(1), datetime64[us](1

In [23]:
test_final_merged["Year"] = test_final_merged["Date"].dt.year
test_final_merged["Month"] = test_final_merged["Date"].dt.month
test_final_merged["Day"] = test_final_merged["Date"].dt.day
test_final_merged["Week_Of_Year"] = test_final_merged["Date"].dt.isocalendar().week.astype(int)
test_final_merged["Week_Of_Month"] = ((test_final_merged["Date"].dt.day-1) // 7 + 1)

In [24]:
test_final_merged.head()

,Store,Dept,Date,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Type,Size,Year,Month,Day,Week_Of_Year,Week_Of_Month
0,1,1,2012-11-02,False,55.32,3.386,6766.44,5147.70,50.82,3639.90,2737.42,223.462779,6.573,A,151315,2012,11,2,44,1
1,1,1,2012-11-09,False,61.24,3.314,11421.32,3370.89,40.28,4646.79,6154.16,223.481307,6.573,A,151315,2012,11,9,45,2
2,1,1,2012-11-16,False,52.92,3.252,9696.28,292.10,103.78,1133.15,6612.69,223.512911,6.573,A,151315,2012,11,16,46,3
3,1,1,2012-11-23,True,56.23,3.211,883.59,4.17,74910.32,209.91,303.32,223.561947,6.573,A,151315,2012,11,23,47,4
4,1,1,2012-11-30,False,52.34,3.207,2460.03,NaN,3838.35,150.57,6966.34,223.610984,6.573,A,151315,2012,11,30,48,5


In [25]:
test_final_merged.info()

<class 'pandas.DataFrame'>
RangeIndex: 115064 entries, 0 to 115063
Data columns (total 20 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   Store          115064 non-null  int64         
 1   Dept           115064 non-null  int64         
 2   Date           115064 non-null  datetime64[us]
 3   IsHoliday      115064 non-null  bool          
 4   Temperature    115064 non-null  float64       
 5   Fuel_Price     115064 non-null  float64       
 6   MarkDown1      114915 non-null  float64       
 7   MarkDown2      86437 non-null   float64       
 8   MarkDown3      105235 non-null  float64       
 9   MarkDown4      102176 non-null  float64       
 10  MarkDown5      115064 non-null  float64       
 11  CPI            76902 non-null   float64       
 12  Unemployment   76902 non-null   float64       
 13  Type           115064 non-null  str           
 14  Size           115064 non-null  int64         
 15  Year       

***
EDA
***

In [26]:
train_final_merged.info()

<class 'pandas.DataFrame'>
RangeIndex: 421570 entries, 0 to 421569
Data columns (total 21 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   Store          421570 non-null  int64         
 1   Dept           421570 non-null  int64         
 2   Date           421570 non-null  datetime64[us]
 3   Weekly_Sales   421570 non-null  float64       
 4   IsHoliday      421570 non-null  bool          
 5   Temperature    421570 non-null  float64       
 6   Fuel_Price     421570 non-null  float64       
 7   MarkDown1      150681 non-null  float64       
 8   MarkDown2      111248 non-null  float64       
 9   MarkDown3      137091 non-null  float64       
 10  MarkDown4      134967 non-null  float64       
 11  MarkDown5      151432 non-null  float64       
 12  CPI            421570 non-null  float64       
 13  Unemployment   421570 non-null  float64       
 14  Type           421570 non-null  str           
 15  Size       

In [27]:
train_final_merged.describe()

,Store,Dept,Date,Weekly_Sales,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,MarkDown4,MarkDown5,CPI,Unemployment,Size,Year,Month,Day,Week_Of_Year,Week_Of_Month
count,421570.000000,421570.000000,421570,421570.000000,421570.000000,421570.000000,150681.000000,111248.000000,137091.000000,134967.000000,151432.000000,421570.000000,421570.000000,421570.000000,421570.000000,421570.000000,421570.000000,421570.000000,421570.000000
mean,22.200546,44.260317,2011-06-18 08:30:31.963375,15981.258123,60.090059,3.361027,7246.420196,3334.628621,1439.421384,3383.168256,4628.975079,171.201947,7.960289,136727.915739,2010.968591,6.449510,15.673131,25.826762,2.691691
min,1.000000,1.000000,2010-02-05 00:00:00,-4988.940000,-2.060000,2.472000,0.270000,-265.760000,-29.100000,0.220000,135.160000,126.064000,3.879000,34875.000000,2010.000000,1.000000,1.000000,1.000000,1.000000
25%,11.000000,18.000000,2010-10-08 00:00:00,2079.650000,46.680000,2.933000,2240.270000,41.600000,5.080000,504.220000,1878.440000,132.022667,6.891000,93638.000000,2010.000000,4.000000,8.000000,14.000000,2.000000
50%,22.000000,37.000000,2011-06-17 00:00:00,7612.030000,62.090000,3.452000,5347.450000,192.000000,24.600000,1481.310000,3359.450000,182.318780,7.866000,140167.000000,2011.000000,6.000000,16.000000,26.000000,3.000000
75%,33.000000,74.000000,2012-02-24 00:00:00,20205.852500,74.280000,3.738000,9210.900000,1926.940000,103.990000,3595.040000,5563.800000,212.416993,8.572000,202505.000000,2012.000000,9.000000,23.000000,38.000000,4.000000
max,45.000000,99.000000,2012-10-26 00:00:00,693099.360000,100.140000,4.468000,88646.760000,104519.540000,141630.610000,67474.850000,108519.280000,227.232807,14.313000,219622.000000,2012.000000,12.000000,31.000000,52.000000,5.000000
std,12.785297,30.492054,NaN,22711.183519,18.447931,0.458515,8291.221345,9475.357325,9623.078290,6292.384031,5962.887455,39.159276,1.863296,60980.583328,0.796876,3.243217,8.753549,14.151887,1.263852


Let's make some plots to understand our data better;

We start by plotting all stores total weekly sales

In [28]:
total_weekly_sales = train_final_merged.groupby('Week_Of_Year', as_index=False)["Weekly_Sales"].sum()
total_weekly_sales.head()

,Week_Of_Year,Weekly_Sales
0,1,8.773121e+07
1,2,8.269676e+07
2,3,8.273564e+07
3,4,7.943483e+07
4,5,1.419895e+08


In [29]:
import plotly.express as px

fig = px.line(
    total_weekly_sales,
    x="Week_Of_Year",
    y="Weekly_Sales",
    title="Total Walmart Stores Sales by Weeks of the Year",
    labels={
        "Week_Of_Year": "Week of the Year",
        "Weekly_Sales": "Total Weekly Sales ($)"
    },
)

fig.update_traces(
    hovertemplate="Week %{x}<br>Sales: $%{y:,.0f}<extra></extra>"
)

fig.update_layout(
    template="plotly_white",
    hovermode="x",
    width=800,
    height=500,
    title_x = 0.5
)

fig.show()


We can see that sales peak in week 7, 14, 22, 27, 40, 47 and 51 and drop really low at weeks 44, 48 and 52

Now we plot average store sales per week

In [30]:
weekly_store_sales = (train_final_merged.groupby(["Week_Of_Year", "Store"], as_index=False)["Weekly_Sales"].sum())
weekly_store_sales.head()


,Week_Of_Year,Store,Weekly_Sales
0,1,1,2995102.20
1,1,2,3557570.93
2,1,3,776419.55
3,1,4,3910242.34
4,1,5,598425.97


In [31]:
avg_store_sales = (weekly_store_sales.groupby("Week_Of_Year", as_index=False)["Weekly_Sales"].mean())
avg_store_sales.head()

,Week_Of_Year,Weekly_Sales
0,1,1.949582e+06
1,2,1.837706e+06
2,3,1.838570e+06
3,4,1.765218e+06
4,5,3.155321e+06


In [32]:
import plotly.express as px

fig = px.line(
    avg_store_sales,
    x="Week_Of_Year",
    y="Weekly_Sales",
    title="Average Walmart Store Sales by Week of Year",
    labels={
        "Week_Of_Year": "Week of the Year",
        "Weekly_Sales": "Average Weekly Store Sales ($)"
    },
)

fig.update_traces(
    hovertemplate="Week %{x}<br>Avg Store Sales: $%{y:,.0f}<extra></extra>"
)

fig.update_layout(
    template="plotly_white",
    hovermode="x",
    width=1000,
    height=500,
    title_x=0.5
)

fig.show()


Now lets see if total sales increase or decrease from 2010 to 2012

In [33]:
yearly_sales = (train_final_merged.groupby("Year")["Weekly_Sales"].sum().reset_index())
yearly_sales.head()

,Year,Weekly_Sales
0,2010,2.288886e+09
1,2011,2.448200e+09
2,2012,2.000133e+09


In [34]:
fig = px.line(yearly_sales, 
              x="Year", 
              y="Weekly_Sales", 
              title="Walmart Total Sales Trend by Year (2010-2012)",
              labels={
                    "Year": "Year",
                    "Weekly_Sales": "Total Sales ($)"
                    },)

fig.update_traces(
    hovertemplate="Year: %{x}<br>Total Sales: $%{y:,.0f}<extra></extra>"
)

fig.update_layout(
    template="plotly_white",
    hovermode="x",
    width=1000,
    height=500,
    title_x=0.5
)


we can see that sales increased from 2010 to 2011 (app. 160M$) and decreased heavily from 2011 and 2012 (app. 450M$)

Now let's plot the sales of each store type (A,B,C) over 3 years

In [35]:
type_sales = (train_final_merged.groupby(["Week_Of_Year","Type"])["Weekly_Sales"].sum().reset_index())
type_sales.head()

,Week_Of_Year,Type,Weekly_Sales
0,1,A,56214517.08
1,1,B,25517191.19
2,1,C,5999501.45
3,2,A,53389134.89
4,2,B,23382012.79


In [36]:
fig = px.line(type_sales, 
              x="Week_Of_Year", 
              y="Weekly_Sales", 
              title="Sales by Week Over the Year for each Store Type",
              color= "Type",
              labels={
                    "Week_Of_Year": "Week of the Year",
                    "Weekly_Sales": "Total Sales ($)"
                    },)

fig.update_traces(
    hovertemplate="Week: %{x}<br>Total Sales: $%{y:,.0f}<extra></extra>"
)

fig.update_layout(
    template="plotly_white",
    hovermode="x",
    width=1000,
    height=500,
    title_x=0.5
)


We observe that Type C stores show relatively stable, linear sales throughout the year, whereas Type A and B stores display similar seasonal patterns: 

Their sales rise sharply around week 4, peak even more strongly at week 51, and then drop significantly in weeks 52 and 43.


Let's see the effect of Holidays on the weekly sales:

In [37]:
week_counts = (
    train_final_merged[["Date", "IsHoliday"]]
    .drop_duplicates()
    .groupby("IsHoliday")
    .size()
)

week_counts

IsHoliday
False    133
True      10
dtype: int64

In [38]:
holiday_effect = (train_final_merged.groupby("IsHoliday", as_index=False)["Weekly_Sales"].mean())
holiday_effect

,IsHoliday,Weekly_Sales
0,False,15901.445069
1,True,17035.823187


In [39]:
fig = px.bar(
    holiday_effect,
    x="IsHoliday",
    y="Weekly_Sales",
    title="Impact of Holidays on Weekly Sales",
    labels={
        "IsHoliday": "Week Type",
        "Weekly_Sales": "Average Weekly Sales ($)"
    },
    text_auto=".2s"
)

fig.update_traces(
    hovertemplate="%{x}<br>Avg Sales: $%{y:,.0f}<extra></extra>"
)

fig.update_layout(
    template="plotly_white",
    width=600,
    height=450,
    title_x=0.5
)

fig.show()


Holiday weeks generate higher sales than non-holiday weeks.
On average, weekly sales during holidays (17,035) are higher than during non-holiday weeks (15,901).

(17035.823187 − 15901.445069) / 15901.445069 × 100 = 7.13%

Even though holiday weeks are rare (10 out of 143), they still boost sales by approximately 7% compared to regular weeks, which represents a substantial impact on revenue for Walmart.



Markdowns and sales

In [40]:
markdown_cols = ["MarkDown1","MarkDown2","MarkDown3","MarkDown4","MarkDown5"]

weekly_markdown_sales = (
    train_final_merged
    .groupby("Week_Of_Year", as_index=False)
    .agg({
        "Weekly_Sales": "sum",
        **{c: "sum" for c in markdown_cols}
    })
)

weekly_markdown_sales["Total_MarkDown"] = (
    weekly_markdown_sales[markdown_cols].sum(axis=1)
)


In [41]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=weekly_markdown_sales["Week_Of_Year"],
    y=weekly_markdown_sales["Weekly_Sales"],
    name="Total Sales",
    yaxis="y1",
    line=dict(width=3)
))

fig.add_trace(go.Scatter(
    x=weekly_markdown_sales["Week_Of_Year"],
    y=weekly_markdown_sales["Total_MarkDown"],
    name="Total MarkDown",
    yaxis="y2",
    line=dict(width=3, dash="dot")
))

fig.update_layout(
    title="Total Sales vs Total MarkDowns by Week of Year",
    title_x=0.5,
    xaxis_title="Week of Year",
    yaxis=dict(title="Total Sales ($)"),
    yaxis2=dict(
        title="Total MarkDown ($)",
        overlaying="y",
        side="right"
    ),
    template="plotly_white",
    hovermode="x",
    width=1000,
    height=500
)

fig.show()


Let's see that in more details

In [42]:
markdown_cols = ["MarkDown1","MarkDown2","MarkDown3","MarkDown4","MarkDown5"]

weekly_markdown_sales = (
    train_final_merged
    .groupby("Week_Of_Year", as_index=False)
    .agg({
        "Weekly_Sales": "sum",
        **{c: "sum" for c in markdown_cols}
    })
)


In [43]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=weekly_markdown_sales["Week_Of_Year"],
    y=weekly_markdown_sales["Weekly_Sales"],
    name="Total Sales",
    yaxis="y1",
    line=dict(width=4)
))

for md in markdown_cols:
    fig.add_trace(go.Scatter(
        x=weekly_markdown_sales["Week_Of_Year"],
        y=weekly_markdown_sales[md],
        name=md,
        yaxis="y2",
        line=dict(width=2, dash="dot")
    ))

fig.update_layout(
    title="Weekly Sales vs MarkDown Campaigns",
    title_x=0.5,
    xaxis_title="Week of the Year",

    yaxis=dict(
        title="Total Sales ($)",
        showgrid=True
    ),

    yaxis2=dict(
        title="MarkDown Amount ($)",
        overlaying="y",
        side="right",
        showgrid=False
    ),

    template="plotly_white",
    hovermode="x",
    width=1100,
    height=550
)

fig.show()


We can see that when Markdowns spike, the sales also spike  at the same time or just right after

Let's plot the year to year sales differances:

In [44]:
sales_year_week = (
    train_final_merged
    .groupby(["Year", "Week_Of_Year"], as_index=False)["Weekly_Sales"]
    .sum()
)


In [45]:
import plotly.express as px

fig = px.line(
    sales_year_week,
    x="Week_Of_Year",
    y="Weekly_Sales",
    color="Year",
    title="Walmart Weekly Sales by Year",
    labels={
        "Week_Of_Year": "Week of Year",
        "Weekly_Sales": "Total Weekly Sales ($)"
    }
)

fig.update_traces(
    hovertemplate="Year %{Year}<br>Week %{x}<br>Sales: $%{y:,.0f}<extra></extra>"
)

fig.update_layout(
    template="plotly_white",
    hovermode="x unified",
    width=1100,
    height=550,
    title_x=0.5
)

fig.show()


Correlation matrix:

Let's make a correlation matrix to understand better the relationships between attributes

But first, we need to fid Nan in Markdowns with a 0, since it means there was no promotion


In [46]:
markdown_cols = ["MarkDown1", "MarkDown2", "MarkDown3", "MarkDown4", "MarkDown5"]

train_final_merged[markdown_cols] = train_final_merged[markdown_cols].fillna(0)
test_final_merged[markdown_cols] = test_final_merged[markdown_cols].fillna(0)


In [47]:
train_final_merged.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,...,MarkDown5,CPI,Unemployment,Type,Size,Year,Month,Day,Week_Of_Year,Week_Of_Month
0,1,1,2010-02-05,24924.50,False,42.31,2.572,0.0,0.0,0.0,...,0.0,211.096358,8.106,A,151315,2010,2,5,5,1
1,1,1,2010-02-12,46039.49,True,38.51,2.548,0.0,0.0,0.0,...,0.0,211.242170,8.106,A,151315,2010,2,12,6,2
2,1,1,2010-02-19,41595.55,False,39.93,2.514,0.0,0.0,0.0,...,0.0,211.289143,8.106,A,151315,2010,2,19,7,3
3,1,1,2010-02-26,19403.54,False,46.63,2.561,0.0,0.0,0.0,...,0.0,211.319643,8.106,A,151315,2010,2,26,8,4
4,1,1,2010-03-05,21827.90,False,46.50,2.625,0.0,0.0,0.0,...,0.0,211.350143,8.106,A,151315,2010,3,5,9,1


In [48]:
numeric_cols = train_final_merged.select_dtypes(include=[np.number])
corr_matrix = numeric_cols.corr()
corr_matrix["Weekly_Sales"]

Store           -0.085195
Dept             0.148032
Weekly_Sales     1.000000
Temperature     -0.002312
Fuel_Price      -0.000120
MarkDown1        0.047172
MarkDown2        0.020716
MarkDown3        0.038562
MarkDown4        0.037467
MarkDown5        0.050465
CPI             -0.020921
Unemployment    -0.025864
Size             0.243828
Year            -0.010111
Month            0.028409
Day             -0.006187
Week_Of_Year     0.027673
Week_Of_Month   -0.004004
Name: Weekly_Sales, dtype: float64

In [49]:
fig = px.imshow(
    corr_matrix,
    text_auto="0.2f",
    aspect="auto",
    color_continuous_scale="RdBu",
    title="Correlation Matrix of Walmart Features"
)

fig.update_layout(
    width=1000,
    height=900,
    title_x=0.5
)

fig.show()


This correlation between attributes shows that store characteristics, dominate Walmart sales far more than external economic or weather factors. The strongest relationship with weekly sales is store size (0.24), confirming that larger stores consistently generate higher revenue regardless of timing or conditions. Department (0.15) is also highly influential, reflecting the fact that different product categories (e.g., groceries, electronics, clothing) naturally operate at very different sales scales. Promotional activity has a clear positive impact: all MarkDown variables have positive correlations (around 0.02–0.05), indicating that when Walmart runs discounts or special offers, sales reliably increase, even though the effect varies by promotion type. In contrast, macroeconomic and environmental variables such as temperature, fuel price, CPI, and unemployment show almost no correlation, suggesting that Walmart’s weekly demand is very stable and not strongly affected by short-term economic or weather fluctuations.

Now, let's add some meaningfull data: 

We will add labor day week, thanksgiving week, chrismas week, super bowl


In [ ]:

THANKSGIVING_WEEK = {
    2010: pd.Timestamp("2010-11-26"),
    2011: pd.Timestamp("2011-11-25"),
    2012: pd.Timestamp("2012-11-23"),
    2013: pd.Timestamp("2013-11-29"),  
}

CHRISTMAS_WEEK = {
    2010: pd.Timestamp("2010-12-31"),
    2011: pd.Timestamp("2011-12-30"),
    2012: pd.Timestamp("2012-12-28"),
    2013: pd.Timestamp("2013-12-27"),  
}

SUPER_BOWL_WEEK = {
    2010: pd.Timestamp("2010-02-12"),
    2011: pd.Timestamp("2011-02-11"),
    2012: pd.Timestamp("2012-02-10"),
    2013: pd.Timestamp("2013-02-08"),  
}

LABOR_DAY_WEEK = {
    2010: pd.Timestamp("2010-09-10"),
    2011: pd.Timestamp("2011-09-09"),
    2012: pd.Timestamp("2012-09-07"),
    2013: pd.Timestamp("2013-09-06"),  
}

In [51]:
def add_days_to_holidays(df):
    df = df.copy()
    df["Days_to_Thanksgiving"] = df["Date"].apply(lambda d: (THANKSGIVING_WEEK[d.year] - d).days).astype(int)
    df["Days_to_Christmas"]    = df["Date"].apply(lambda d: (CHRISTMAS_WEEK[d.year] - d).days).astype(int)
    df["Days_to_SuperBowl"]     = df["Date"].apply(lambda d: (SUPER_BOWL_WEEK[d.year] - d).days).astype(int)
    df["Days_to_LaborDay"]      = df["Date"].apply(lambda d: (LABOR_DAY_WEEK[d.year] - d).days).astype(int)
    return df

In [52]:
train_final_merged = add_days_to_holidays(train_final_merged)
test_final_merged  = add_days_to_holidays(test_final_merged)


In [53]:
train_final_merged[["Date","Days_to_SuperBowl","Days_to_LaborDay","Days_to_Thanksgiving","Days_to_Christmas"]].head()

,Date,Days_to_SuperBowl,Days_to_LaborDay,Days_to_Thanksgiving,Days_to_Christmas
0,2010-02-05,7,217,294,329
1,2010-02-12,0,210,287,322
2,2010-02-19,-7,203,280,315
3,2010-02-26,-14,196,273,308
4,2010-03-05,-21,189,266,301


let's see the correlation matrix again:

In [54]:
numeric_cols = train_final_merged.select_dtypes(include=[np.number])
corr_matrix = numeric_cols.corr()
corr_matrix["Weekly_Sales"]

Store                  -0.085195
Dept                    0.148032
Weekly_Sales            1.000000
Temperature            -0.002312
Fuel_Price             -0.000120
MarkDown1               0.047172
MarkDown2               0.020716
MarkDown3               0.038562
MarkDown4               0.037467
MarkDown5               0.050465
CPI                    -0.020921
Unemployment           -0.025864
Size                    0.243828
Year                   -0.010111
Month                   0.028409
Day                    -0.006187
Week_Of_Year            0.027673
Week_Of_Month          -0.004004
Days_to_Thanksgiving   -0.027673
Days_to_Christmas      -0.027673
Days_to_SuperBowl      -0.027673
Days_to_LaborDay       -0.027673
Name: Weekly_Sales, dtype: float64

In [55]:
fig = px.imshow(
    corr_matrix,
    text_auto="0.2f",
    aspect="auto",
    color_continuous_scale="Jet",
    title="Correlation Matrix of Walmart Features"
)

fig.update_layout(
    width=1000,
    height=900,
    title_x=0.5
)

fig.show()

In [56]:
train_final_merged.head()

,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,...,Size,Year,Month,Day,Week_Of_Year,Week_Of_Month,Days_to_Thanksgiving,Days_to_Christmas,Days_to_SuperBowl,Days_to_LaborDay
0,1,1,2010-02-05,24924.50,False,42.31,2.572,0.0,0.0,0.0,...,151315,2010,2,5,5,1,294,329,7,217
1,1,1,2010-02-12,46039.49,True,38.51,2.548,0.0,0.0,0.0,...,151315,2010,2,12,6,2,287,322,0,210
2,1,1,2010-02-19,41595.55,False,39.93,2.514,0.0,0.0,0.0,...,151315,2010,2,19,7,3,280,315,-7,203
3,1,1,2010-02-26,19403.54,False,46.63,2.561,0.0,0.0,0.0,...,151315,2010,2,26,8,4,273,308,-14,196
4,1,1,2010-03-05,21827.90,False,46.50,2.625,0.0,0.0,0.0,...,151315,2010,3,5,9,1,266,301,-21,189


preprocessing:

In [57]:
from sklearn.preprocessing import OrdinalEncoder

Ordinal_encoder = OrdinalEncoder(categories=[["A","B","C"]])

train_final_merged["Type_Encoded"] = Ordinal_encoder.fit_transform(train_final_merged[["Type"]])
test_final_merged["Type_encoded"]  = Ordinal_encoder.transform(test_final_merged[["Type"]])

In [ ]:
train_final_merged["IsHoliday"] = train_final_merged["IsHoliday"].astype(int)
test_final_merged["IsHoliday"]  = test_final_merged["IsHoliday"].astype(int)

In [59]:
train_final_merged.sample()

,Store,Dept,Date,Weekly_Sales,IsHoliday,Temperature,Fuel_Price,MarkDown1,MarkDown2,MarkDown3,...,Year,Month,Day,Week_Of_Year,Week_Of_Month,Days_to_Thanksgiving,Days_to_Christmas,Days_to_SuperBowl,Days_to_LaborDay,Type_Encoded
223895,23,59,2010-12-24,22343.33,0,22.96,3.15,0.0,0.0,0.0,...,2010,12,24,51,4,-28,7,-315,-105,1.0


Modeling

now, we are going to split the training set into 2, the set of training and one of validation. (train on 2011 and below and validate on testing set)

In [ ]:
train_data = train_final_merged[train_final_merged["Year"] <= 2011]
val_data   = train_final_merged[train_final_merged["Year"] == 2012]

print("Train rows:", train_data.shape)
print("Validation rows:", val_data.shape)


Train rows: (294132, 26)
Validation rows: (127438, 26)


we start by training on catboostmodel

In [62]:
from catboost import CatBoostRegressor

train_final_merged["Type"] = train_final_merged["Type"].astype(str)
val_data["Type"] = val_data["Type"].astype(str)

features = [
    "Store","Dept","Type","Size",
    "Temperature","Fuel_Price","CPI","Unemployment",
    "MarkDown1","MarkDown2","MarkDown3","MarkDown4","MarkDown5",
    "Year","Month","Week_Of_Year","Week_Of_Month",
    "Days_to_SuperBowl","Days_to_LaborDay",
    "Days_to_Thanksgiving","Days_to_Christmas",
    "IsHoliday"
]

X_train = train_data[features]
y_train = train_data["Weekly_Sales"]
X_val   = val_data[features]
y_val   = val_data["Weekly_Sales"]



cat_model = CatBoostRegressor(
    iterations=4000,
    learning_rate=0.05,
    depth=8,
    loss_function="RMSE",
    eval_metric="RMSE",
    random_seed=42,
    verbose=200,
    task_type="GPU",
    devices="0"
)

cat_model.fit(
    X_train, y_train,
    cat_features=["Store","Dept","Type"],
    eval_set=(X_val, y_val),
    early_stopping_rounds=200,
    use_best_model=True
)


0:	learn: 22232.1102068	test: 21352.8821300	best: 21352.8821300 (0)	total: 30ms	remaining: 1m 59s
200:	learn: 9497.5378535	test: 9027.7235021	best: 9027.7235021 (200)	total: 3.79s	remaining: 1m 11s
400:	learn: 8265.3379162	test: 7904.0137700	best: 7904.0137700 (400)	total: 7.77s	remaining: 1m 9s
600:	learn: 7711.7463020	test: 7387.3457348	best: 7387.3457348 (600)	total: 11.8s	remaining: 1m 6s
800:	learn: 7322.5961155	test: 7050.4301205	best: 7050.4301205 (800)	total: 15.8s	remaining: 1m 3s
1000:	learn: 7021.8989131	test: 6819.0433913	best: 6819.0433913 (1000)	total: 20.2s	remaining: 1m
1200:	learn: 6787.9948979	test: 6636.3001729	best: 6636.3001729 (1200)	total: 24.9s	remaining: 58s
1400:	learn: 6610.0568834	test: 6511.4970676	best: 6511.4970676 (1400)	total: 30.1s	remaining: 55.8s
1600:	learn: 6452.1597865	test: 6405.9829245	best: 6405.9829245 (1600)	total: 34.8s	remaining: 52.2s
1800:	learn: 6300.5057379	test: 6314.0760822	best: 6314.0760822 (1800)	total: 39.8s	remaining: 48.6s
2000:

In [67]:
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error

pred = cat_model.predict(X_val)

rmse = np.sqrt(mean_squared_error(y_val, pred))
mae  = mean_absolute_error(y_val, pred)

w = np.where(X_val["IsHoliday"].astype(bool).values, 5.0, 1.0)
wmae = np.sum(w * np.abs(y_val.values - pred)) / np.sum(w)

print("RMSE:", rmse)
print("MAE:", mae)
print("WMAE:", wmae)



RMSE: 5931.437901034397
MAE: 2975.38192962152
WMAE: 3013.600255497437


The CatBoost regression model achieved an RMSE of 5,931, MAE of 2,975, and WMAE of 3,014 on the validation set. This indicates that the model’s weekly sales predictions differ from the true values by approximately $3,000 per store–department–week on average. The higher RMSE compared to MAE suggests that a small number of weeks—likely associated with holidays, promotions, or high-volume departments—produce larger errors. The WMAE, which penalizes holiday weeks more heavily, is slightly higher than the MAE, showing that the model still finds it more difficult to capture sharp sales spikes during special periods. Overall, the model successfully learns the general structure and seasonality of the data, providing a strong baseline for further improvement.